# https://pbpython.com/styling-pandas.html

In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_excel('data/2018_Sales_Total.xlsx')

In [3]:
df.head(5)

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2018-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2018-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2018-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2018-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2018-01-01 23:26:55


In [4]:
# quick summary to see how much the customers have purchased from us and what their average purchase amount looks like
df.groupby('name')['ext price'].agg(['mean', 'sum'])

,mean,sum
name,,
Barton LLC,1334.615854,109438.50
"Cronin, Oberbrunner and Spencer",1339.321642,89734.55
"Frami, Hills and Schmidt",1438.466528,103569.59
"Fritsch, Russel and Anderson",1385.366790,112214.71
"Halvorson, Crona and Champlin",1206.971724,70004.36
Herman LLC,1336.532258,82865.00
Jerde-Hilpert,1265.072247,112591.43
"Kassulke, Ondricka and Metz",1350.797969,86451.07
Keeling LLC,1363.977027,100934.30


In [5]:
(df.groupby('name')['ext price'].agg(['mean', 'sum']).style.format('${0:,.2f}')) # style format https://mkaz.blog/code/python-string-format-cookbook/

,mean,sum
name,,
Barton LLC,"$1,334.62","$109,438.50"
"Cronin, Oberbrunner and Spencer","$1,339.32","$89,734.55"
"Frami, Hills and Schmidt","$1,438.47","$103,569.59"
"Fritsch, Russel and Anderson","$1,385.37","$112,214.71"
"Halvorson, Crona and Champlin","$1,206.97","$70,004.36"
Herman LLC,"$1,336.53","$82,865.00"
Jerde-Hilpert,"$1,265.07","$112,591.43"
"Kassulke, Ondricka and Metz","$1,350.80","$86,451.07"
Keeling LLC,"$1,363.98","$100,934.30"


In [6]:
#  look at total sales by each month
monthly_sales = df.groupby([pd.Grouper(key='date', freq='M')])['ext price'].agg(['sum']).reset_index()
monthly_sales['pct_of_total'] = monthly_sales['sum'] / df['ext price'].sum()
print(monthly_sales)

         date        sum  pct_of_total
0  2018-01-31  185361.66      0.091818
1  2018-02-28  146211.62      0.072426
2  2018-03-31  203921.38      0.101012
3  2018-04-30  174574.11      0.086475
4  2018-05-31  165418.55      0.081940
5  2018-06-30  174089.33      0.086235
6  2018-07-31  191662.11      0.094939
7  2018-08-31  153778.59      0.076174
8  2018-09-30  168443.17      0.083438
9  2018-10-31  171495.32      0.084950
10 2018-11-30  119961.22      0.059423
11 2018-12-31  163867.26      0.081171


In [7]:
# use a dictionary to define a unique formatting string for each column
format_dict = {'sum':'${0:,.0f}', 'date': '{:%m-%Y}', 'pct_of_total': '{:.2%}'}
monthly_sales.style.format(format_dict).hide_index() # hide_index function suppresses the display of the index


date,sum,pct_of_total
01-2018,"$185,362",9.18%
02-2018,"$146,212",7.24%
03-2018,"$203,921",10.10%
04-2018,"$174,574",8.65%
05-2018,"$165,419",8.19%
06-2018,"$174,089",8.62%
07-2018,"$191,662",9.49%
08-2018,"$153,779",7.62%
09-2018,"$168,443",8.34%
10-2018,"$171,495",8.49%


In [8]:
# highlight the highest number in green and the lowest number in color Trinidad (#cd4f39)
(monthly_sales
 .style
 .format(format_dict)
 .hide_index()
 .highlight_max(subset=['sum','pct_of_total'],color='lightgreen')
 .highlight_min(subset=['sum','pct_of_total'],color='#cd4f39'))

date,sum,pct_of_total
01-2018,"$185,362",9.18%
02-2018,"$146,212",7.24%
03-2018,"$203,921",10.10%
04-2018,"$174,574",8.65%
05-2018,"$165,419",8.19%
06-2018,"$174,089",8.62%
07-2018,"$191,662",9.49%
08-2018,"$153,779",7.62%
09-2018,"$168,443",8.34%
10-2018,"$171,495",8.49%


In [9]:
# background_gradient can highlight the range of values in a column
(monthly_sales.style
 .format(format_dict)
 .background_gradient(subset=['sum'], cmap='BuGn')) # https://matplotlib.org/stable/tutorials/colors/colormaps.html

,date,sum,pct_of_total
0,01-2018,"$185,362",9.18%
1,02-2018,"$146,212",7.24%
2,03-2018,"$203,921",10.10%
3,04-2018,"$174,574",8.65%
4,05-2018,"$165,419",8.19%
5,06-2018,"$174,089",8.62%
6,07-2018,"$191,662",9.49%
7,08-2018,"$153,779",7.62%
8,09-2018,"$168,443",8.34%
9,10-2018,"$171,495",8.49%


In [10]:
# drawing bar charts within the columns
(monthly_sales
 .style
 .format(format_dict)
 .hide_index()
 .bar(color='#FFA07A', vmin=100_000, subset=['sum'], align='zero')
 .bar(color='lightgreen', vmin=0, subset=['pct_of_total'], align='zero')
 .set_caption('2018 Sales Performance'))

date,sum,pct_of_total
01-2018,"$185,362",9.18%
02-2018,"$146,212",7.24%
03-2018,"$203,921",10.10%
04-2018,"$174,574",8.65%
05-2018,"$165,419",8.19%
06-2018,"$174,089",8.62%
07-2018,"$191,662",9.49%
08-2018,"$153,779",7.62%
09-2018,"$168,443",8.34%
10-2018,"$171,495",8.49%


In [11]:
import sparklines

def sparkline_str(x):
    bins=np.histogram(x)[0]
    sl = ''.join(sparklines(bins))
    return sl
sparkline_str.__name__ = "sparkline"

In [12]:
df.groupby('name')['quantity', 'ext price'].agg(['mean', sparkline_str])

/var/folders/3r/gzbp9xfx0612n9bv_qj8rkqw0000gn/T/ipykernel_41951/3732561847.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby('name')['quantity', 'ext price'].agg(['mean', sparkline_str])
/usr/local/Caskroom/miniconda/base/envs/mach_lern/lib/python3.9/site-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


ValueError: no results